In [39]:
raw = 'https://gist.githubusercontent.com/jwalsh/ce1dc0436aba5b7a5c9666f47fa5a380/raw/5ce3854392b43ff97907112d344fc008229b0445/titanic.csv'


In [31]:
# Create DataFrames
import pandas as pd
from pyspark.sql import SparkSession, Row
from pyspark.sql import toPandas
from datetime import datetime, date
import numpy as np

In [145]:
# инициализация спарка
spark = SparkSession.builder.getOrCreate()

In [ ]:
df = spark.createDataFrame(
    [
        Row(a=1, b=2, c='str1', d=date(2022, 1, 1), e=datetime(2022, 1, 1, 12, 0)),
        Row(a=3, b=3, c='str1', d=date(2022, 1, 1), e=datetime(2022, 1, 1, 12, 0)),
        Row(a=4, b=5, c='str1', d=date(2022, 1, 1), e=datetime(2022, 1, 1, 12, 0)),
    ]
)
df

In [38]:
# Создайте PySpark DataFrame из pandas DataFrame
panda_df = pd.DataFrame({
    'a': [1, 2, 3],
    'b': [2., 3., 4.],
    'c': ['string1', 'string2', 'string3'],
    'd': [date(2000, 1, 1), date(2000, 2, 1), date(2000, 3, 1)],
    'e': [datetime(2000, 1, 1, 12, 0), datetime(2000, 1, 2, 12, 0), datetime(2000, 1, 3, 12, 0)]
})
df = spark.createDataFrame(panda_df)
df

a,b,c,d,e
1,2.0,string1,2000-01-01,2000-01-01 12:00:00
2,3.0,string2,2000-02-01,2000-01-02 12:00:00
3,4.0,string3,2000-03-01,2000-01-03 12:00:00


In [26]:
# Создайте PySpark DataFrame из RDD, состоящего из списка кортежей.

rdd = spark.sparkContext.parallelize([
    (1, 2., 'string1', date(2000, 1, 1), datetime(2000, 1, 1, 12, 0)),
    (2, 3., 'string2', date(2000, 2, 1), datetime(2000, 1, 2, 12, 0)),
    (3, 4., 'string3', date(2000, 3, 1), datetime(2000, 1, 3, 12, 0))
])
df = spark.createDataFrame(rdd, schema=['a', 'b', 'c', 'd', 'e'])
df

a,b,c,d,e
1,2.0,string1,2000-01-01,2000-01-01 12:00:00
2,3.0,string2,2000-02-01,2000-01-02 12:00:00
3,4.0,string3,2000-03-01,2000-01-03 12:00:00


In [9]:
# with schema
df = spark.createDataFrame([
    (1, 2., 'string1', date(2000, 1, 1), datetime(2000, 1, 1, 12, 0)),
    (2, 3., 'string2', date(2000, 2, 1), datetime(2000, 1, 2, 12, 0)),
    (3, 4., 'string3', date(2000, 3, 1), datetime(2000, 1, 3, 12, 0))
], schema='a long, b double, c string, d date, e timestamp')
df

DataFrame[a: bigint, b: double, c: string, d: date, e: timestamp]

In [146]:
# создать дф на своих данных
from pathlib import Path
path = f'{Path.home()}/Desktop/learning/learn_spark/Data_learn'
df = spark.read.format('csv').options(header=False)\
     .load(f'{path}/pandas/titanic.csv')

df

DataFrame[_c0: string, _c1: string, _c2: string, _c3: string, _c4: string, _c5: string, _c6: string, _c7: string, _c8: string, _c9: string, _c10: string, _c11: string, _c12: string, _c13: string]

In [147]:
df.head()

Row(_c0='0', _c1='1', _c2='0', _c3='3', _c4='Braund', _c5=' Mr. Owen Harris', _c6='male', _c7='22.0', _c8='1', _c9='0', _c10='A/5 21171', _c11='7.25', _c12=None, _c13='S')

In [148]:
df.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: string (nullable = true)
 |-- _c7: string (nullable = true)
 |-- _c8: string (nullable = true)
 |-- _c9: string (nullable = true)
 |-- _c10: string (nullable = true)
 |-- _c11: string (nullable = true)
 |-- _c12: string (nullable = true)
 |-- _c13: string (nullable = true)



### просмотр данных

In [149]:
df.show(1, vertical=True)

-RECORD 0----------------
 _c0  | 0                
 _c1  | 1                
 _c2  | 0                
 _c3  | 3                
 _c4  | Braund           
 _c5  |  Mr. Owen Harris 
 _c6  | male             
 _c7  | 22.0             
 _c8  | 1                
 _c9  | 0                
 _c10 | A/5 21171        
 _c11 | 7.25             
 _c12 | null             
 _c13 | S                
only showing top 1 row



In [76]:
# Кроме того, вы можете включить spark.sql.repl.eagerEval.enabled настройку для быстрой оценки PySpark DataFrame 
# в таких ноутбуках, как Jupyter. Количество отображаемых
#  строк можно контролировать с помощью spark.sql.repl.eagerEval.maxNumRows конфигурации.
spark.conf.set('spark.sql.repl.eagerEval.enabled', True)
df


_c0,_c1,_c2,_c3,_c4,_c5,_c6,_c7,_c8,_c9,_c10,_c11,_c12,_c13
0,1,0,3,Braund,Mr. Owen Harris,male,22.0,1,0,A/5 21171,7.25,null,S
1,2,1,1,Cumings,Mrs. John Bradle...,female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,Heikkinen,Miss. Laina,female,26.0,0,0,STON/O2. 3101282,7.925,null,S
3,4,1,1,Futrelle,Mrs. Jacques Hea...,female,35.0,1,0,113803,53.1,C123,S
4,5,0,3,Allen,Mr. William Henry,male,35.0,0,0,373450,8.05,null,S
5,6,0,3,Moran,Mr. James,male,null,0,0,330877,8.4583,null,Q
6,7,0,1,McCarthy,Mr. Timothy J,male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,Palsson,Master. Gosta Le...,male,2.0,3,1,349909,21.075,null,S
8,9,1,3,Johnson,Mrs. Oscar W (El...,female,27.0,0,2,347742,11.1333,null,S
9,10,1,2,Nasser,Mrs. Nicholas (A...,female,14.0,1,0,237736,30.0708,null,C


In [79]:
# имена столбов можно увидеть
print(df.columns)

['_c0', '_c1', '_c2', '_c3', '_c4', '_c5', '_c6', '_c7', '_c8', '_c9', '_c10', '_c11', '_c12', '_c13']


In [150]:
# Сводка по ДФ
df.select('_c2','_c6').describe().show()

+-------+-------------------+------+
|summary|                _c2|   _c6|
+-------+-------------------+------+
|  count|                156|   156|
|   mean|0.34615384615384615|  null|
| stddev|0.47727514420450834|  null|
|    min|                  0|female|
|    max|                  1|  male|
+-------+-------------------+------+



In [86]:
# DataFrame.collect() собирает распределенные данные на стороне драйвера как локальные данные в Python.
# Обратите внимание, что это может вызвать ошибку нехватки памяти, когда набор данных слишком велик, 
# чтобы поместиться на стороне драйвера, поскольку он собирает все данные от исполнителей на стороне драйвера.

df.take(1)

[Row(_c0='0', _c1='1', _c2='0', _c3='3', _c4='Braund', _c5=' Mr. Owen Harris', _c6='male', _c7='22.0', _c8='1', _c9='0', _c10='A/5 21171', _c11='7.25', _c12=None, _c13='S')]

In [151]:
df.select(df._c4).show(1, vertical=True)
df.show(5)

-RECORD 0-----
 _c4 | Braund 
only showing top 1 row

+---+---+---+---+---------+--------------------+------+----+---+---+----------------+-------+----+----+
|_c0|_c1|_c2|_c3|      _c4|                 _c5|   _c6| _c7|_c8|_c9|            _c10|   _c11|_c12|_c13|
+---+---+---+---+---------+--------------------+------+----+---+---+----------------+-------+----+----+
|  0|  1|  0|  3|   Braund|     Mr. Owen Harris|  male|22.0|  1|  0|       A/5 21171|   7.25|null|   S|
|  1|  2|  1|  1|  Cumings| Mrs. John Bradle...|female|38.0|  1|  0|        PC 17599|71.2833| C85|   C|
|  2|  3|  1|  3|Heikkinen|         Miss. Laina|female|26.0|  0|  0|STON/O2. 3101282|  7.925|null|   S|
|  3|  4|  1|  1| Futrelle| Mrs. Jacques Hea...|female|35.0|  1|  0|          113803|   53.1|C123|   S|
|  4|  5|  0|  3|    Allen|   Mr. William Henry|  male|35.0|  0|  0|          373450|   8.05|null|   S|
+---+---+---+---+---------+--------------------+------+----+---+---+----------------+-------+----+----+
only showi

In [152]:
# докинем еще один столб
from pyspark.sql.functions import upper
df.withColumn('new_col', upper(df._c5)).show(5)

+---+---+---+---+---------+--------------------+------+----+---+---+----------------+-------+----+----+--------------------+
|_c0|_c1|_c2|_c3|      _c4|                 _c5|   _c6| _c7|_c8|_c9|            _c10|   _c11|_c12|_c13|             new_col|
+---+---+---+---+---------+--------------------+------+----+---+---+----------------+-------+----+----+--------------------+
|  0|  1|  0|  3|   Braund|     Mr. Owen Harris|  male|22.0|  1|  0|       A/5 21171|   7.25|null|   S|     MR. OWEN HARRIS|
|  1|  2|  1|  1|  Cumings| Mrs. John Bradle...|female|38.0|  1|  0|        PC 17599|71.2833| C85|   C| MRS. JOHN BRADLE...|
|  2|  3|  1|  3|Heikkinen|         Miss. Laina|female|26.0|  0|  0|STON/O2. 3101282|  7.925|null|   S|         MISS. LAINA|
|  3|  4|  1|  1| Futrelle| Mrs. Jacques Hea...|female|35.0|  1|  0|          113803|   53.1|C123|   S| MRS. JACQUES HEA...|
|  4|  5|  0|  3|    Allen|   Mr. William Henry|  male|35.0|  0|  0|          373450|   8.05|null|   S|   MR. WILLIAM HENRY|


In [164]:
# присвоение имен колонкам через новый документы
headers = [
    'df_id',
    'PassengerId',
    'Survived',
    'Pclass',
    'Lname',
    'Name',
    'Sex',
    'Age',
    'SibSp',
    'Parch',
    'Ticket',
    'Fare',
    'Cabin',
    'Embarked'
]

pd_df = df.toPandas()
pd_df.columns = headers
pd_df.to_csv('for_spark_titanic.csv', header=True)


In [192]:
df = spark.read.format('csv').options(header=True) .load('for_spark_titanic.csv')
# через распаковку сносим не нужные столбы
tables = ['_c0', 'df_id']
df = df.drop(*tables)

df.show(10)


+-----------+--------+------+---------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|    Lname|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+---------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|   Braund|     Mr. Owen Harris|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|  Cumings| Mrs. John Bradle...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen|         Miss. Laina|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1| Futrelle| Mrs. Jacques Hea...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|    Allen|   Mr. William Henry|  male|35.0|    0|    0|          373450|   8.05| null|       S|


In [193]:
df.schema.fields

[StructField(PassengerId,StringType,true),
 StructField(Survived,StringType,true),
 StructField(Pclass,StringType,true),
 StructField(Lname,StringType,true),
 StructField(Name,StringType,true),
 StructField(Sex,StringType,true),
 StructField(Age,StringType,true),
 StructField(SibSp,StringType,true),
 StructField(Parch,StringType,true),
 StructField(Ticket,StringType,true),
 StructField(Fare,StringType,true),
 StructField(Cabin,StringType,true),
 StructField(Embarked,StringType,true)]

### Филтьтрация

In [201]:
df.filter(df.Pclass == 3).count()

96

### Group By

In [209]:
print(df.schema.names)
df.groupby('Survived').count().show()


['PassengerId', 'Survived', 'Pclass', 'Lname', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked']
+--------+-----+
|Survived|count|
+--------+-----+
|       0|  102|
|       1|   54|
+--------+-----+



### Работа с SQL 
DataFrame и Spark SQL используют один и тот же механизм выполнения, поэтому их можно беспрепятственно использовать взаимозаменяемо. Например, вы можете зарегистрировать DataFrame как таблицу и легко запустить SQL, как показано ниже:

In [225]:
df.createOrReplaceTempView('titanic')
quare = spark.sql('SELECT Lname, Name, PassengerId, Survived FROM titanic\
    WHERE Survived = 1 ').show()

+-------------+--------------------+-----------+--------+
|        Lname|                Name|PassengerId|Survived|
+-------------+--------------------+-----------+--------+
|      Cumings| Mrs. John Bradle...|          2|       1|
|    Heikkinen|         Miss. Laina|          3|       1|
|     Futrelle| Mrs. Jacques Hea...|          4|       1|
|      Johnson| Mrs. Oscar W (El...|          9|       1|
|       Nasser| Mrs. Nicholas (A...|         10|       1|
|    Sandstrom| Miss. Marguerite...|         11|       1|
|      Bonnell|     Miss. Elizabeth|         12|       1|
|      Hewlett| Mrs. (Mary D Kin...|         16|       1|
|     Williams|  Mr. Charles Eugene|         18|       1|
|   Masselmani|         Mrs. Fatima|         20|       1|
|      Beesley|        Mr. Lawrence|         22|       1|
|      McGowan|          Miss. Anna|         23|       1|
|       Sloper| Mr. William Thom...|         24|       1|
|      Asplund| Mrs. Carl Oscar ...|         26|       1|
|      O'Dwyer

In [223]:
spark.sql('select count(PassengerId) from titanic where Survived = 1').show()

+------------------+
|count(PassengerId)|
+------------------+
|                54|
+------------------+



In [228]:
from pyspark.sql import functions as F
# теперь тоже самое, но уже используя pysqark

df.filter(F.col('Survived') == 1)\
    .select(F.col("PassengerId"), F.col('Lname'), F.col('Lname')).show(10)

+-----------+----------+----------+
|PassengerId|     Lname|     Lname|
+-----------+----------+----------+
|          2|   Cumings|   Cumings|
|          3| Heikkinen| Heikkinen|
|          4|  Futrelle|  Futrelle|
|          9|   Johnson|   Johnson|
|         10|    Nasser|    Nasser|
|         11| Sandstrom| Sandstrom|
|         12|   Bonnell|   Bonnell|
|         16|   Hewlett|   Hewlett|
|         18|  Williams|  Williams|
|         20|Masselmani|Masselmani|
+-----------+----------+----------+
only showing top 10 rows



In [229]:
spark.stop()